In [1]:
# attempt at resuming from a terminated run (originally trained with transformers v 4.2.1)
# problems 1) Trainer.train(resume_from_checkpoint = xx) only came out after v4.2.1 (ver. I originally used.)
#            So, I updated the transformers to .. v4.26.1.. Behaviors might be slightly different between two versions.
#          2) In the github, there's statement that you need to install transformers >= 4.5.1. 
#            But I originally used v4.2.1. The training might not replicate Sachin's experiments.
# --> 2 is critical. 
# ----> Should I retrain the model from scratch?
# ----> If so, better write up a continuous training script and use it...T.T

In [2]:
# #train the classifier
# python -u examples/training_constraint_models/train_classifier.py\
#     data/toxicity/jigsaw-unintended-bias-in-toxicity-classification\
#     0,1\
#     train\
#     dev\
#     test\
#     roberta-base\
#     models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds\
#     gpt2-roberta full gpt2-large freeze-vecmap dontbinarize jsonl

In [3]:
params = ['', 'data/toxicity/jigsaw-unintended-bias-in-toxicity-classification',
 '0,1',
 'train',
 'dev',
 'test',
 'roberta-base',
 'models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds',
 'gpt2-roberta',
 'full',
 'gpt2-large',
 'freeze-vecmap',
 'dontbinarize',
 'jsonl']

In [4]:
import torch
import os
import json

import transformers
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AddedToken
from torch import optim

import numpy as np

In [5]:
# os.makeirs(params[7], exist_ok=True)

def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [6]:
base_path = params[1]
binarize_labels = False
if len(params) > 12:
    binarize_labels = params[12] == "binarize_labels"

filetype = "txt"
if len(params) > 13:
    filetype = params[13]

labels = [int(label) for label in params[2].split(",")]
train_paths = []
valid_paths = []
test_paths = []
for label in labels:
    train_paths.append(open(f"{base_path}/{params[3]}_{label}.{filetype}"))
    valid_paths.append(open(f"{base_path}/{params[4]}_{label}.{filetype}"))
    test_paths.append(open(f"{base_path}/{params[5]}_{label}.{filetype}"))

def create_dataset(paths, labelses):
    texts, labels = [], []
    # print(paths)
    for i, path in enumerate(paths):
        for l in path:
            if filetype == "jsonl":
                text = json.loads(l)["text"]
            else:
                text = l.strip()
            if binarize_labels:
                label = 0
                if labelses[i] <= 2:
                    label = 0
                    texts.append(text)
                    labels.append(label)
                elif labelses[i] >= 3:
                    label = 1
                    texts.append(text)
                    labels.append(label)
            else:
                labels.append(labelses[i])
                texts.append(text)
            
    print("create_dataset", len(texts), len(labels), set(labels))
    return texts, labels
    
train_texts, train_labels = create_dataset(train_paths, labels)
val_texts, val_labels = create_dataset(valid_paths, labels)
test_texts, test_labels = create_dataset(test_paths, labels)

create_dataset 311564 311564 {0, 1}
create_dataset 4000 4000 {0, 1}
create_dataset 4000 4000 {0, 1}


In [7]:
# tokenizer = AutoTokenizer.from_pretrained(params[6], cache_dir="hf_cache")
# config = AutoConfig.from_pretrained(params[6], cache_dur="hf_cache", num_labels=len(labels))

tokenizer_ = AutoTokenizer.from_pretrained(params[6], cache_dir="hf_cache")
if params[10] != "none":
    tokenizer = AutoTokenizer.from_pretrained(params[10], cache_dir="hf_cache")
    tokenizer.model_max_length = min(tokenizer_.model_max_length, tokenizer.model_max_length)
else:
    tokenizer = tokenizer_
    # tokenizer = AutoTokenizer.from_pretrained(params[6], cache_dir="hf_cache")

In [8]:
config = AutoConfig.from_pretrained(params[6], cache_dur="hf_cache", num_labels=len(labels))
config2 = None
if params[10] != "none":  
    config2 = AutoConfig.from_pretrained(params[10], cache_dur="hf_cache", num_labels=len(labels))
    print(config2.pad_token_id)
    config2.pad_token_id = tokenizer.pad_token_id
    print(config2.pad_token_id)
    print("look above for padding")

    tokenizer_ = AutoTokenizer.from_pretrained(params[6], config=config)
    tokenizer.model_max_length = min(tokenizer_.model_max_length, tokenizer.model_max_length)

# config.n_positions = max_length
# config.max_position_embeddings = max_length

None
None
look above for padding


In [9]:
# if params[8] == "krishna":
#     SPECIAL_TOKENS = {
#         "additional_special_tokens": ["<dense-vectors>", "<tokens>", "<verb>", "<ARG0>", "<ARG1>", "<global-dense-vectors>"],
#         "pad_token": "<eos>",
#         "bos_token": "<bos>",
#         "eos_token": "<eos>"
#     }
#     print("Adding special tokens")
#     tokenizer.add_special_tokens(SPECIAL_TOKENS)
#     config.pad_token_id = tokenizer.pad_token_id

# elif params[8] == "roberta" :
#     SPECIAL_TOKENS = {"bos_token": "<s>", "eos_token": "</s>", "pad_token": "<pad>"}
#     print("Adding special tokens")
#     tokenizer.add_special_tokens(SPECIAL_TOKENS)
#     config.pad_token_id = tokenizer.pad_token_id

# elif params[8] == "dialogpt": 
#     SPECIAL_TOKENS = {"pad_token": tokenizer.eos_token}
#     print("Adding special tokens")
#     tokenizer.add_special_tokens(SPECIAL_TOKENS)
#     config.pad_token_id = tokenizer.pad_token_id

# elif params[8] == "gpt2":
#     SPECIAL_TOKENS = {"pad_token": tokenizer.eos_token}
#     config.pad_token_id = tokenizer.eos_token_id
#     # tokenizer.pad_token_id = tokenizer.eos_token_id
#     print("Adding special tokens")
#     tokenizer.add_special_tokens(SPECIAL_TOKENS)
#     print(tokenizer)

# elif params[8] == "bert":
#     # SPECIAL_TOKENS = {"pad_token": tokenizer.eos_token}
#     # config.pad_token_id = tokenizer.eos_token_id
#     # print("Adding special tokens")
#     # tokenizer.add_special_tokens(SPECIAL_TOKENS)
#     pass

# elif params[8] == "gpt2-roberta":
if params[8] == "gpt2-roberta":
    SPECIAL_TOKENS = {"pad_token": tokenizer.eos_token}
    # config.pad_token_id = tokenizer.eos_token_id
    print("Adding special tokens")
    tokenizer.add_special_tokens(SPECIAL_TOKENS)

# elif params[8] == "gpt2-distilbert":
#     SPECIAL_TOKENS = {"pad_token": tokenizer.eos_token}
#     # config.pad_token_id = tokenizer.eos_token_id
#     print("Adding special tokens")
#     tokenizer.add_special_tokens(SPECIAL_TOKENS)


tokenizer.save_pretrained(f"{params[7]}/checkpoint_best")

Adding special tokens


('models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/tokenizer_config.json',
 'models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/special_tokens_map.json',
 'models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/vocab.json',
 'models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/merges.txt',
 'models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/added_tokens.json',
 'models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/tokenizer.json')

In [10]:
# if params[9] != "only_tokenizer": # all below are inside this if statement

print("tokenizer loaded")
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
print(test_encodings['input_ids'][0], len(test_encodings['input_ids'][0]))
# input()
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)
test_dataset = Dataset(test_encodings, test_labels)

print("datasets loaded and tokenizer")

tokenizer loaded
[26287, 5398, 6641, 5057, 318, 379, 16677, 4, 357, 11275, 83, 14, 12519, 352, 13, 22, 1058, 352, 1267, 530, 286, 262, 4511, 287, 262, 995, 0, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5

In [11]:
# if params[10] != "none":
model = AutoModelForSequenceClassification.from_pretrained(params[10], config=config2) # unindented
# model.resize_token_embeddings(len(tokenizer))

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
#         if params[11] == "random":  
#             embeds = model.get_input_embeddings()
#             embeds.weight.data.normal_(mean=0.0, std=0.02)
#             if embeds.padding_idx is not None:
#                 embeds.weight.data[module.padding_idx].zero_()
#             model.resize_token_embeddings(len(tokenizer))

#         elif params[11] == "freeze":
#             model = AutoModelForSequenceClassification.from_pretrained(params[6], config=config)
#             embeds = model.get_input_embeddings()
#             for p in embeds.parameters():
#                 p.requires_grad = False
#             # embeds.requires_grad=False
#             model.resize_token_embeddings(len(tokenizer))

#         elif params[11] == "freeze-project":
#             embeds = model.get_input_embeddings()
#             new_embeds = torch.nn.Embedding(embeds.num_embeddings, embeds.embedding_dim)
#             for p in new_embeds.parameters():
#                 p.requires_grad = False
#             # new_embeds.requires_grad = False
#             new_embeds.weight.data.copy_(embeds.weight)
#             print(model.device)
#             config.new_n_embd = new_embeds.embedding_dim
#             config.new_vocab_size = new_embeds.num_embeddings
#             # if params[8] == "gpt2-roberta":
#             #     config.pad_token_id = tokenizer.eos_token_id
#             model_ = AutoModelForSequenceClassification.from_pretrained(params[6], config=config)
#             new_embeds = torch.nn.Sequential(new_embeds, torch.nn.Linear(new_embeds.embedding_dim, model_.get_input_embeddings().embedding_dim, bias=False))
#             model_.set_input_embeddings(new_embeds)
#             model = model_

#         elif params[11] == "freeze-eye":
#             embeds = model.get_input_embeddings()
#             new_embeds = torch.nn.Embedding(embeds.num_embeddings, embeds.embedding_dim)
#             for p in new_embeds.parameters():
#                 p.requires_grad = False
#             # new_embeds.requires_grad = False
#             new_embeds.weight.data.copy_(embeds.weight)
#             print(model.device)
#             config.new_n_embd = new_embeds.embedding_dim
#             # if params[8] == "gpt2-roberta":
#             #     config.pad_token_id = tokenizer.eos_token_id
#             model_ = AutoModelForSequenceClassification.from_pretrained(params[6], config=config)
#             eye = torch.nn.Linear(new_embeds.embedding_dim, model_.get_input_embeddings().embedding_dim, bias=False)
#             eye.weight.data.copy_(torch.eye(new_embeds.embedding_dim).data)
#             new_embeds = torch.nn.Sequential(new_embeds, eye)
#             model_.set_input_embeddings(new_embeds)
#             model = model_

#         elif params[8] == "gpt2-roberta": # part below unindented
def learn_vecmap(X, y):
    print("computing vecmap")
    w = torch.inverse(X.t().matmul(X)).matmul(X.t()).matmul(y)
    vecmap = torch.nn.Linear(w.size(0), w.size(1), bias=False)
    print(w.size(), vecmap.weight.size())
    vecmap.weight.data.copy_(w.data.t())
    return vecmap

def vocab_permutation(vocab1, vocab2):
    vocab2itos = {k:v for v,k in vocab2.items()}
    vocab2list = [vocab2itos[k] for k in range(len(vocab2itos))]

    perm1 = []
    perm2 = []
    unincluded = []
    for i, word in enumerate(vocab2list):
        if word in vocab1:
            perm1.append(vocab1[word])
            perm2.append(i)
        else:
            unincluded.append(word)

    print(unincluded)
    return perm1, perm2

embeds = model.get_input_embeddings()
new_embeds = torch.nn.Embedding(embeds.num_embeddings, embeds.embedding_dim)
for p in new_embeds.parameters():
    p.requires_grad = False

new_embeds.weight.data.copy_(embeds.weight)
print(model.device)
config.new_n_embd = new_embeds.embedding_dim
config.new_vocab_size = new_embeds.num_embeddings

cpu


In [13]:
model_ = AutoModelForSequenceClassification.from_pretrained(params[6], config=config)
tokenizer_ = AutoTokenizer.from_pretrained(params[6], config=config)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [14]:
perm, perm_ = vocab_permutation(tokenizer.vocab, tokenizer_.vocab)
old_embeds = model_.get_input_embeddings()
vecmap = learn_vecmap(new_embeds.weight[perm], old_embeds.weight[perm_])
new_embeds = torch.nn.Sequential(new_embeds, vecmap)
model_.set_input_embeddings(new_embeds)
model = model_

# else:
#     model =  AutoModelForSequenceClassification.from_pretrained(params[6], config=config)
#     model.resize_token_embeddings(len(tokenizer))
# # model =  AutoModelForSequenceClassification.from_pretrained(params[6], config=config)

['<s>', '<pad>', '</s>', '<unk>', 'madeupword0000', 'madeupword0001', 'madeupword0002', '<mask>']
computing vecmap
torch.Size([1280, 768]) torch.Size([768, 1280])


In [15]:
mod_path = '/home/hyeryungson/mucoco/models_bak/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds'
# state_dict
mod = torch.load(os.path.join(mod_path, 'results/checkpoint-76000/pytorch_model.bin'))

In [16]:
model.load_state_dict(mod)

<All keys matched successfully>

In [17]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(model.device)
os.makedirs(params[7], exist_ok=True)

cuda:0


In [18]:
## optimizer 
# need to separate parameters into 2 groups to apply different weight decay values (AdamW)
# weight decay only applied to parameter weights (not to bias, layernorm weight/bias)
# code source: https://discuss.huggingface.co/t/adamw-pytorch-vs-huggingface/30562
# why?: weight decay = for overfitting https://forums.fast.ai/t/is-weight-decay-applied-to-the-bias-term/73212

no_decay = ["bias", "LayerNorm.weight", "LayerNorm.bias"]
optimizer_grouped_parameters = [
            {
                "params": [
                    p
                    for n, p in model.named_parameters()
                    if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.01,
            },
            {
                "params": [
                    p
                    for n, p in model.named_parameters()
                    if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]
optimizer = optim.AdamW(
            optimizer_grouped_parameters,
            lr=1e-05,
            eps=1e-08,
            weight_decay=0
)

In [19]:
# optim state_dict
opt = torch.load(os.path.join(mod_path, 'results/checkpoint-76000/optimizer.pt'))

In [20]:
optimizer.load_state_dict(opt)

In [21]:
lr_sch = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=600, num_training_steps=194720, last_epoch=3)

In [22]:
## learning rate scheduler
# "linear"
sch = torch.load(os.path.join(mod_path, 'results/checkpoint-76000/scheduler.pt'))

In [23]:
lr_sch.load_state_dict(sch)

In [24]:
lr_sch

In [25]:
# for i in opt['state']:
#     print(i, opt['state'][i]['exp_avg'].size())

In [26]:
# for i, (name, t) in enumerate(model.named_parameters()):
#     if i == 0: 
#         continue
#     if ('bias' in name) or ('LayerNorm' in name):
#         continue
#     print(i, name, ':::' , t.size())

In [27]:
training_args = TrainingArguments(
    output_dir=f'{params[7]}/results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
#     warmup_steps=600,
#     weight_decay=0.01,               # strength of weight decay
#     learning_rate=1e-5,
    logging_dir=f'{params[7]}/logs',            # directory for storing logs
    logging_steps=100,
    evaluation_strategy="steps",
    save_total_limit=1,
    eval_steps=500,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
    report_to="wandb"
)

print(training_args.n_gpu)

1


In [28]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_sch)
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
train_result = trainer.train(resume_from_checkpoint=True)

Loading model from models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-76000.
You are resuming training from a checkpoint trained with 4.2.1 of Transformers but your current version is 4.26.1. This is not recommended and could yield to errors or unwanted behaviors.
***** Running training *****
  Num examples = 311564
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 194720
  Number of trainable parameters = 87026690
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 3
  Continuing training from global step 76000
  Will skip the first 3 epochs then the first 70336 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/70336 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: hayleyson (ski-ml). Use `wandb login --relogin` to force relogin


...

Step,Training Loss,Validation Loss,Accuracy
76500,0.082600,0.297577,0.937000
77000,0.117400,0.304370,0.930750
77500,0.134600,0.299758,0.930500
78000,0.139000,0.277251,0.933750
78500,0.114900,0.335345,0.930750
79000,0.116500,0.298580,0.932750
79500,0.123800,0.290136,0.933500
80000,0.109600,0.318558,0.928250
80500,0.106700,0.306804,0.935500
81000,0.101200,0.314432,0.932750


***** Running Evaluation *****
  Num examples = 4000
  Batch size = 4
Saving model checkpoint to models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-76500
Configuration saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-76500/config.json
Model weights saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-76500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 4
Saving model checkpoint to models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-78000
Configuration saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-78000/config.json
Model weights saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-78000/pytorch_model.bin
Deleting older checkpoint [models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-

Configuration saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-84000/config.json
Model weights saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-84000/pytorch_model.bin
Deleting older checkpoint [models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-83500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 4
Saving model checkpoint to models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-84500
Configuration saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-84500/config.json
Model weights saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-84500/pytorch_model.bin
Deleting older checkpoint [models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-84000] d

Deleting older checkpoint [models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-90000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 4
Saving model checkpoint to models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-91000
Configuration saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-91000/config.json
Model weights saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-91000/pytorch_model.bin
Deleting older checkpoint [models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-90500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 4
Saving model checkpoint to models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-91500
Configuration saved in models/roberta-base-jig

Configuration saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-110000/config.json
Model weights saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-110000/pytorch_model.bin
Deleting older checkpoint [models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-109500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 4
Saving model checkpoint to models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-110500
Configuration saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-110500/config.json
Model weights saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-110500/pytorch_model.bin
Deleting older checkpoint [models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/results/checkpoint-11

In [23]:
print("training finished")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/hyeryungson/anaconda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 311564
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 194720
  Number of trainable parameters = 87026690
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


AssertionError: 

In [112]:
os.makedirs(f"{params[7]}/tmp")

In [113]:
trainer.save_model(output_dir=f"{params[7]}/tmp") 
print("model saved")

Saving model checkpoint to models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/tmp
Configuration saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/tmp/config.json
Model weights saved in models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/tmp/pytorch_model.bin


model saved


In [11]:
print("running evaluation now")

metrics = trainer.evaluate(val_dataset)
print("validation", metrics)
metrics = trainer.evaluate(test_dataset)
print("test", metrics)

create_dataset 311564 311564 {0, 1}
create_dataset 4000 4000 {0, 1}
create_dataset 4000 4000 {0, 1}
None
None
look above for padding
Adding special tokens
tokenizer loaded
[26287, 5398, 6641, 5057, 318, 379, 16677, 4, 357, 11275, 83, 14, 12519, 352, 13, 22, 1058, 352, 1267, 530, 286, 262, 4511, 287, 262, 995, 0, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cpu


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

['<s>', '<pad>', '</s>', '<unk>', 'madeupword0000', 'madeupword0001', 'madeupword0002', '<mask>']
computing vecmap
torch.Size([1280, 768]) torch.Size([768, 1280])
cuda:0
2
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can e

...

wandb: Currently logged in as: hayleyson (ski-ml). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/hyeryungson/anaconda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


KeyboardInterrupt: 